# Chapter 4. 수치형 데이터 다루기

## 4.0 소개

* 정량적 데이터 : 반별 학생 수, 월간 매출, 시험 점수 같이 어떤 것을 측량한 결과
    * 자연스럽게 숫자로 표현된다. 이를 머신러닝에 맞은 특성으로 변환하는 다양한 전략을 소개한다.

## 4.1 특성 스케일 바꾸기
* 사이킷런의 MinMaxScaler로 특성 배열의 스케일 조정

In [13]:
import numpy as np
from sklearn import preprocessing

feature = np.array([[-500.5],
                   [-100.1],
                   [0],
                   [100.1],
                   [900.9]])

# 먼저 스케일러 객체를 만들어야 한다.
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

# 특성의 스케일을 변환시킨다.
scaled_feature = minmax_scale.fit_transform(feature)

# 특성을 출력한다.
scaled_feature

array([[0.        ],
       [0.28571429],
       [0.35714286],
       [0.42857143],
       [1.        ]])

* 모든 특성은 동일한 스케일을 가진다고 가정한다.
    * 주로 0 ~ 1 혹은 -1 ~ 1 사이이다.
* 스케일 조정 기법
    * 최소-최대 스케일링(min-max scaling) : 특성의 최솟값과 최댓값을 사용해 일정 범위 안으로 값을 조정
    
$$x'_i = {{x_i - \min(x)}\over{\max(x)-\min(x)}}$$

* $x$ : 특성 벡터, $x_i$ : 특성 x의 개별 원소, $x'_i$ : 스케일이 바뀐 원소

* MinMaxScaler : 특성 스케일을 위한 두 가지 방법
    * fit 메서드 : 특성의 최솟값과 최댓값을 계산 후 transform 메서드로 특성의 스케일 조정
    * fit_transform 메서드 : 두 연산을 한번에 처리한다.
    * **동일한 두 변환을 다른 데이터 셋에 적용** : fit과 transform을 따로 호출해야 한다.

In [14]:
# 훈련 세트를 변환
preprocessing.MinMaxScaler().fit_transform(feature[:3])

array([[0. ],
       [0.8],
       [1. ]])

In [15]:
# 테스트 셋 변환
preprocessing.MinMaxScaler().fit_transform(feature[3:])

array([[0.],
       [1.]])

* 훈련 세트와 테스트 세트를 가각 변환하면 서로 다른 비율로 데이터를 변환한다.
    * 위에 처럼 **데이터가 다른 스케일로 변환되면 훈련 세트에서 학습한 모델을 테스트 세트에서 사용할 수 없다.**
    * 그래서 훈련 세트에서 학습한 변환기를 가지고 변환시키는 것이 좋습니다.

In [16]:
# 훈련 셋으로 변환기를 학습한다.
scaler = preprocessing.MinMaxScaler().fit(feature[:3])
scaler.transform(feature[:3])

array([[0. ],
       [0.8],
       [1. ]])

In [17]:
# 훈련 셋으로 학습한 변환기로 테스트 셋 변환한다.
scaler.transform(feature[3:])

array([[1.2],
       [2.8]])

## 4.2 특성을 표준화하기
* StandardScaler : 특성을 평균이 0이고 표준편차가 1이 되도록 변환한다.

In [24]:
import numpy as np
from sklearn import preprocessing

x = np.array([[-1000.1],
            [-200.2],
            [500.5],
            [600.6],
            [9000.9]])

scaler = preprocessing.StandardScaler()

standardized = scaler.fit_transform(x)

standardized

array([[-0.76058269],
       [-0.54177196],
       [-0.35009716],
       [-0.32271504],
       [ 1.97516685]])

* 표준 정규분포로 근사하는 스케일링 방식
* 표준화 사용 : 데이터 평균 $\bar{x} = 0$이고 표준편차$\sigma = 1$이 되도록 변환

$$ x'_i = {{x_i-\bar{x}}\over{\sigma}}$$

* 주성분 분석 : 표준화
* 신경망 : 최소-최대 스케일링

In [23]:
print("평균:", round(standardized.mean()))
print("표준편차:", round(standardized.std()))

평균: 0.0
표준편차: 1.0


* RobustScaler : 중간값과 사분위 범위를 사용해서 특성의 스케일을 조정한다.
    * 데이터에 이상치가 많다면 특성의 평균과 표준편차에 영향을 미친다.
    * 표준화에 부정적인 영향을 주므로 이럴 때 RobustScaler로 이상치 영향을 벗어나자.

In [25]:
# 변환 객체 제작
robust_scaler = preprocessing.RobustScaler()

# 특성 변환
robust_scaler.fit_transform(x)

array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

In [26]:
# IQR(interquartile range)
# robustcaler는 데이터에서 중간값을 빼고 IQR로 나눈다.
interquartile_range = x[3] - x[1]
(x - np.median(x)) / interquartile_range

array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

* QuantileTransformer : 훈련 데이터를 1000개의 분위로 나누어 0과 1 사이에 고르게 분포시킨다.
    * 이상치 영향을 줄인다.
    * 해결에 나온 특성 x는 5개 특성이므로 0,25,50,75,100% 위치에 할당한다.

In [28]:
preprocessing.QuantileTransformer().fit_transform(x)

C:\Users\gurdk\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (5). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


array([[0.  ],
       [0.25],
       [0.5 ],
       [0.75],
       [1.  ]])

## 4.3 정규화하기
* 샘플의 특성값을 전체 길이가 1인 단위 노름(Norm) 변환
* Normalizer + norm 이용

In [29]:
import numpy as np
from sklearn.preprocessing import Normalizer

# 특성행렬
features = np.array([[0.5, 0.5],
                    [1.1, 3.4],
                    [1.5, 20.2],
                    [1.63, 34.4],
                    [10.9, 3.3]])

# 변환 객체를 제작한다.
normalizer = Normalizer(norm="l2")

# 이제 변환한다. 주의할 점은 여기서는 fit_transform이 아니라 바로 transform을 쓴다.
normalizer.transform(features)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

* Normalizer : 단위 노름(길이의 합이 1)이 되도록 개별 샘플의 값을 변환한다.
    * 노름 옵션
    * L2 노름
    
    $$\lVert x \rVert_2 = \sqrt{x_{1}^2 + x_{2}^2 + \cdots + x_{n}^2}$$

In [30]:
features_l2_norm = Normalizer(norm="l2").transform(features)

In [31]:
features_l2_norm

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

* 맨해튼(Manhattan) 노름(L1) 지정

$$\lVert{x}\rVert_1 = \sum_{i=1}^n|x_i|$$

In [32]:
# 특성 행렬을 변환한다.
features_l1_norm = Normalizer(norm="l1").transform(features)

features_l1_norm

array([[0.5       , 0.5       ],
       [0.24444444, 0.75555556],
       [0.06912442, 0.93087558],
       [0.04524008, 0.95475992],
       [0.76760563, 0.23239437]])

* norm='l1' : 각 샘플 특성값의 합을 1로 만든다.

In [33]:
# 합 출력
print("첫 번째 샘플값의 합:", features_l1_norm[0, 0] + features_l1_norm[0, 1])

첫 번째 샘플값의 합: 1.0


* Normalizer 행 단위로 변환되어 fit 메서드는 아무런 작업을 수행하지 않는다.
    * 바로 transform 메서드를 사용한다.
    * 'l1', 'l2' 옵션의 변환은 각 행의 L1 노름과 L2 노름을 구해 나눈다.

In [35]:
# L1 노름으로 변환
# 각 행을 합한 결과가 2차원 배열로 유지되게 keepdims=True 설정
features / np.sum(np.abs(features), axis=1, keepdims=True) # axis=1(행)을 열방향으로 더한다.

array([[0.5       , 0.5       ],
       [0.24444444, 0.75555556],
       [0.06912442, 0.93087558],
       [0.04524008, 0.95475992],
       [0.76760563, 0.23239437]])

In [36]:
# L2 노름을 사용한 변환
features / np.sqrt(np.sum(np.square(features), axis=1, keepdims=True))

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

* Normalizer의 norm 매개변수에 'max' 옵션도 지정할 수 있다.
    * 단순히 각 행의 최댓값으로 행의 값을 나눈다.

In [37]:
# 각 행에서 최댓값으로 나눈다.
Normalizer(norm="max").transform(features)

array([[1.        , 1.        ],
       [0.32352941, 1.        ],
       [0.07425743, 1.        ],
       [0.04738372, 1.        ],
       [1.        , 0.30275229]])

## 4.4 다항 특성과 교차항 특성 생성하기
* 다항(polynomial) 특성과 교차항(interaction) 특성을 만든다.
* 사이킷런에 있다.(PolynomialFeatures)

In [38]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

features = np.array([[2, 3],
                    [2, 3],
                    [2, 3]])

# PolynomialFeatures 객체를 만든다.
polynomial_interaction = PolynomialFeatures(degree=2, include_bias=False)

# 다항특성을 만든다.
polynomial_interaction.fit_transform(features)

array([[2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.]])

* degree : 다항식의 최대 차수 결정한다.
    * degree=2 : 2제곱까지 새로운 특성 만든다.
* interaction_only = True : 교차항 특성만 만들 수 있다.
$$x_{1}x_{2}$$

In [39]:
interaction = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
interaction.fit_transform(features)

array([[2., 3., 6.],
       [2., 3., 6.],
       [2., 3., 6.]])

* 특성에 변동 효과를 주입하고자 고차항 특성을 제작한다.($x^2, x^3$등)

* include_bias 매개변수 기본값은 True 입니다.
    * 변환된 특성에 상수항1을 추가한다.

In [40]:
# 상수항 1을 추가한다.
polynomial_bias = PolynomialFeatures(degree=2, include_bias=True).fit(features)
polynomial_bias.transform(features)

array([[1., 2., 3., 4., 6., 9.],
       [1., 2., 3., 4., 6., 9.],
       [1., 2., 3., 4., 6., 9.]])

In [41]:
polynomial_bias.get_feature_names()

['1', 'x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

## 4.5 특성 변환하기

* 사이킷런의 FunctionTransformer 사용해서일련의 특성에 특정 함수를 적용한다.

In [43]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

features = np.array([[2, 3],
                    [2, 3],
                    [2, 3]])

def add_ten(x):
    return x + 10

# 변환기 객체를 설정한다.
ten_transformer = FunctionTransformer(add_ten)

# 특성 행렬을 변환한다.
ten_transformer.transform(features)

array([[12, 13],
       [12, 13],
       [12, 13]])

* 판다스의 apply를 활용할 수도 있다.

In [44]:
import pandas as pd

df = pd.DataFrame(features, columns=["feature_1","feature_2"])

# 함수를 적용한다.
df.apply(add_ten)

C:\Users\gurdk\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,feature_1,feature_2
0,12,13
1,12,13
2,12,13


In [46]:
FunctionTransformer(add_ten, validate=False).transform(np.array([1,2,3]))

array([11, 12, 13])

In [48]:
# 하나 이상 특성에 사용자 정의 함수
from sklearn.compose import ColumnTransformer # 특성 배열이나 데이터프레임 열마다 다른 변환 적용 가능

# feature_1 + 10, feature_2 + 100
# 100 더하는 함수
def add_hundred(x):
    return x+100

ct = ColumnTransformer(
[("add_ten", FunctionTransformer(add_ten, validate=True), ["feature_1"]),
("add_hundred", FunctionTransformer(add_hundred, validate=True), ["feature_2"])])


In [51]:
ct.fit_transform(df)

array([[ 12, 103],
       [ 12, 103],
       [ 12, 103]])

## 4.6 이상치 감지하기
* 일반적인 방법 : 데이터 정규분포 가정 후 타원 그리기

* 타원 안의 샘플(정상치, 레이블 1)이고, 타원 밖의 샘플(이상치, 레이블 -1)로 분류한다.

In [54]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs

features,_ = make_blobs(n_samples=10,
                       n_features=2,
                       centers=1,
                       random_state=1)

In [55]:
# 첫 번째 샘플을 극단 값으로 바꾼다.
features[0,0] = 10000
features[0,1] = 10000

# 이상치 감지 객체 제작
outlier_detector = EllipticEnvelope(contamination=.1)

# 감지 객체 훈련
outlier_detector.fit(features)

# 이상치 예측
outlier_detector.predict(features)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

* contamination : 이상치 비율을 결정하는 매개변수이나 실제로 알지 못한다.
    * 다만 이상치가 적으면, contamination을 작게 설정한다.
    * 이상치가 많다고 판단되면, contamination을 크게 설정한다.

**IQR(사분위 범위) 활용**

In [56]:
feature = features[:,0]

In [57]:
# 이상치 인덱스를 반환하는 함수 제작
def indicies_of_outliers(x):
    q1,q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where(x > upper_bound) | (x < lower_bound)

In [58]:
indicies_of_outliers(feature)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

## 4.7 이상치 다루기
* 이상치 다루는 세 가지 전략

1. 이상치를 삭제한다.

In [59]:
# 이상치 삭제
import pandas as pd

houses = pd.DataFrame()
houses['Price'] = [534433, 392333, 293222, 4322032]
houses['Bathrooms'] = [2,3.5,2,116]
houses['Square_Feet'] = [1500,2500,1500,48000]

In [60]:
# 샘플 필터링
houses[houses['Bathrooms'] < 20]

,Price,Bathrooms,Square_Feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


2. 이상치로 표시하고 이를 특성의 하나로 포함시킨다.(Outlier == 1)

In [61]:
import numpy as np

# 불리언 조건을 기반으로 특성 제작
houses["Outlier"] = np.where(houses["Bathrooms"] < 20, 0, 1)

# 데이터 확인
houses

,Price,Bathrooms,Square_Feet,Outlier
0,534433,2.0,1500,0
1,392333,3.5,2500,0
2,293222,2.0,1500,0
3,4322032,116.0,48000,1


3. 이상치의 영향이 줄어들도록 특성 변환

In [62]:
# 로그 변환
houses["Log_Of_Square_Feet"] = [np.log(x) for x in houses["Square_Feet"]]

houses

,Price,Bathrooms,Square_Feet,Outlier,Log_Of_Square_Feet
0,534433,2.0,1500,0,7.313220
1,392333,3.5,2500,0,7.824046
2,293222,2.0,1500,0,7.313220
3,4322032,116.0,48000,1,10.778956


* 이상치 다룰 때 두 가지 측면 고려
    1. 어떤 것을 이상치로 간주할 것인가?
    2. 다루는 방법이 머신러닝의 목적에 맞아야 한다.
* 마음 속에 있는 최종 목표 : 이상치로 결정하지 않는 게 암묵적인 결정이다.
* 이상치가 있다면 표준화가 적절하지 않다.
    * RobustScaler 처럼 이상치에 민감하지 않은 스케일링 방법을 생각하자.

## 4.8 특성 이산화하기
* 수치 특성을 개별적인 구간으로 나눈다.

1. 임곘값에 따라 특성을 둘로 나눈다.

In [69]:
import numpy as np
from sklearn.preprocessing import Binarizer

age = np.array([[6],[12],[20],[36],[65]])

In [70]:
binarizer = Binarizer(18)

In [71]:
# 특성 변환
binarizer.fit_transform(age)

array([[0],
       [0],
       [1],
       [1],
       [1]])

2. 수치 특성을 여러 임곗값에 따라 나눈다.

In [72]:
np.digitize(age, bins=[20,30,64])

array([[0],
       [0],
       [1],
       [2],
       [3]], dtype=int64)

* bins : 각 구간의 왼쪽 경계값
    * 20까지 구간은 20인 원소 제외하고 이보다 작은 두 개만 포함 된다.
    * 그러면 실질적으로 조건에 맞는 값은, 20 하나가 된다.
    * right = True : 이 동작을 바꿀 수 있다.

In [77]:
np.digitize(age, bins=[20,30,64], right=True)

array([[0],
       [0],
       [0],
       [2],
       [3]], dtype=int64)

In [78]:
# 특성을 나눈다.
np.digitize(age, bins=[18])

array([[0],
       [0],
       [1],
       [1],
       [1]], dtype=int64)

* KBinsDiscretizer 클래스(Scikit-learn 0.20ver) : 연속적인 특성값을 여러 구간으로 나누어준다.
    * 클래스는 나눌 구간 개수를 정한다.

In [81]:
from sklearn.preprocessing import KBinsDiscretizer

kb = KBinsDiscretizer(4, encode='ordinal', strategy='quantile')
kb.fit_transform(age)

array([[0.],
       [1.],
       [2.],
       [3.],
       [3.]])

* strategy 매개변수 기본값 : 'quantile'
    * 각 구간에 포함된 샘플 개수가 비슷하도록 만든다.
    * 'uniform' : 구간의 폭이 동일하도록 만든다.

In [82]:
# 동일한 길이의 구간을 만든다.
kb = KBinsDiscretizer(4, encode='onehot-dense', strategy='uniform')
kb.fit_transform(age)

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

* 구간은 bin_edges_ 속성에서 자세하게 확인할 수 있다.

In [83]:
kb.bin_edges_ 

array([array([ 6.  , 20.75, 35.5 , 50.25, 65.  ])], dtype=object)

## 4.9 군집으로 샘플을 그룹으로 묶기
* 비슷한 샘플을 그룹으로 모으고 싶다.

* K-평균 군집(k-clustering) : 비슷한 샘플을 그룹으로 모을 수 있다.
    * 각 샘플의 소속 그룹이 새로운 특성이 된다.

In [84]:
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# 모의 특성 행렬
features, _ = make_blobs(n_samples=50,
                        n_features=2,
                        centers=3,
                        random_state=1)

In [85]:
dataframe = pd.DataFrame(features, columns=["feature_1", "feature_2"])

clusterer = KMeans(3, random_state=0)

clusterer.fit(features)

# 그룹 소속을 예측한다.
dataframe["group"] = clusterer.predict(features)

dataframe.head(5)

,feature_1,feature_2,group
0,-9.877554,-3.336145,2
1,-7.287210,-8.353986,0
2,-6.943061,-7.023744,0
3,-7.440167,-8.791959,0
4,-6.641388,-8.075888,0


* 뒤에서 군집 알고리즘에 대해 알아본다.
    * 군집이 전처리 단계로 사용할 수 있다.
    * K-평균 같은 비지도 학습 알고리즘을 사용해서 샘플을 그룹으로 모을 수 있다.
    * 최종 목적 : 비슷한 샘플이 모인 그룹을 나타내는 범주형 특성

## 4.10 누락된 값을 가진 샘플을 삭제하기

* 넘파이에서는 한 줄 만으로 간단히 누락된 값이 있는 샘플을 삭제할 수 있다.

* ~ 연산자 : ~np.isnan(dataframes) 조건

In [87]:
import numpy as np

features = np.array([[1.1, 11.1],
                    [2.2, 22.2],
                    [3.3, 33.3],
                    [4.4, 44.4],
                    [np.nan, 55]])

# (~ 연산자) : 누락된 값 없는 샘플만 남겨본다.

features[~np.isnan(features).any(axis=1)]

array([[ 1.1, 11.1],
       [ 2.2, 22.2],
       [ 3.3, 33.3],
       [ 4.4, 44.4]])

* pandas 데이터프레임의 옵션 : dropna()

In [88]:
import pandas as pd

dataframe = pd.DataFrame(features, columns=["feature_1", "feature_2"])

# 누락된 값이 있는 샘플을 제거한다. dropna라는 쉬운 옵션이 있다.
dataframe.dropna()

,feature_1,feature_2
0,1.1,11.1
1,2.2,22.2
2,3.3,33.3
3,4.4,44.4


* 머신러닝 알고리즘은 타깃과 특성 행렬에 있는 결측값은 다룰 수 없다.

* 결측값의 원인에 따라 샘플 삭제 : **데이터의 편향** 증가
* 결측 데이터의 세 가지 종류
    * 완전히 랜덤하게 누락(MCAR) : 값이 누락될 확률이 모든 것에 독립적이다.
    * 랜덤하게 누락(MAR) : 완전히 랜덤하지 않고 다른 특성에서 얻은 정보에 의존한다.(조건부)
    * 랜덤하지 않게 누락(MNAR) : 값이 결측 확률이 랜덤하지 않고 특성에서 잡지 못한 정보에 의존한다.
        * 다른 특성에 의존할 가능성이 높지만, 그 특성에 대한 정보는 제공되지 않는다.

## 4.11 누락된 값 채우기

* 데이터 양이 적으면, KNN(K-nearest neighbors) 알고리즘으로 누락된 값을 예측한다.

In [1]:
import numpy as np
from fancyimpute import KNN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

features, _ = make_blobs(n_samples=1000,
                        n_features=2,
                        random_state=1)

scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# 첫 번째 샘플의 첫 번째 특성을 삭제한다.
true_value = standardized_features[0,0]
standardized_features[0,0] = np.nan

Using TensorFlow backend.


In [3]:
# 특성 행렬에 있는 누락된 값 예측
features_knn_imputed = KNN(k=5, verbose=0).fit_transform(standardized_features)

# 실제 값과 대체된 값 비교
print("실제 값:", true_value)
print("대체된 값:", features_knn_imputed[0,0])

실제 값: 0.8730186113995938
대체된 값: 1.0955332713113226


* 사이킷런 Imputer 모듈로 특성의 평균, 중간값, 최빈값으로 결측값 채우기
    * KNN보다 좋지 않다.

In [10]:
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer()

# 결측값 채우기
features_mean_imputed = mean_imputer.fit_transform(features)

print("실제 값:", true_value)
print("대체된 값:", features_mean_imputed[0,0])

실제 값: 0.8730186113995938
대체된 값: -3.058372724614996


* 결측값 채우는 머신러닝 알고리즘 KNN
    * 가장 가까이 있는 k개의 샘플을 이용해 누락된 값 예측
    * 다섯 개의 이웃한 샘플을 사용해서 누락된 값 예측
    * 단점 : 결측값에 가장 가까운 샘플을 구하고자 결측값과 모든 샘플 사이 거리를 계산해야 한다.
        * 데이터셋 샘플이 수백만개라면 문제가 된다.
* 대안 : 모두 평균값으로 채운다.
    * KNN 처럼 실제 값에 근접하지 않지만 수백만 개의 샘플 데이터셋에 적용 가능하다.
* 값을 대체 전략 사용시, 대체된 값을 가진 샘플인지 여부를 이진 특성으로 만든다.

* SimpleImputer 클래스
    * strategy : mean, median, most_frequent(최빈값, mode), constant(fill_value 매개변수에 지정된 값)